# Import libraries

In [1]:
from dotenv import load_dotenv
import os
from langchain import hub
from langchain_community.document_loaders import ObsidianLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

# Set variables

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Load, split and store embeddings

In [2]:
loader = ObsidianLoader("/Users/danielmcateer/Library/Mobile Documents/iCloud~md~obsidian/Documents/Ideaverse")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant notes from Obsidian

In [3]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

# Run RAG chain using LangChain

In [4]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Get a response from the RAG chain

In [5]:
print(rag_chain.invoke("What are the potential future capabilities of AI?"))

Future capabilities of AI include memory, imagination, planning, search, personalization, and enhanced recommendation systems. Large AI models can exhibit true multi-modality and general system capabilities, leading to transferability in robotics and learning across different domains. Anticipated breakthroughs in AI could bring sudden enhancements in capabilities, potentially exceeding human scientists' pace.


# Cleanup

In [6]:
vectorstore.delete_collection()